In [49]:
import numpy as np
import pandas as pd
from collections import Counter

class MyTreeClf:
    def __init__(self, max_depth=5, min_samples_split=2, max_leafs=20):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_leafs = max_leafs
        self.leafs_cnt = 0  # Начинаем с 0, корень еще не создан
        self.root = None

    def __repr__(self):
        return f"MyTreeClf class: max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, max_leafs={self.max_leafs}"

    def fit(self, X, y):
        self.leafs_cnt = 1
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples = len(X)
        n_labels = len(np.unique(y))

        if (depth >= self.max_depth or
            n_labels == 1 or
            n_samples < self.min_samples_split or
            self.leafs_cnt >= self.max_leafs):
            self.leafs_cnt += 1
            return {'value': self._most_common_label(y)}

        col_name, split_value, ig = self.get_best_split(X, y)

        if ig <= 0:  # Нет прироста информации
            self.leafs_cnt += 1
            return {'value': self._most_common_label(y)}

        left_mask = X[col_name] <= split_value
        right_mask = ~left_mask

        if sum(left_mask) == 0 or sum(right_mask) == 0:
            self.leafs_cnt += 1
            return {'value': self._most_common_label(y)}

        left = self._grow_tree(X[left_mask], y[left_mask], depth + 1)
        right = self._grow_tree(X[right_mask], y[right_mask], depth + 1)

        # Если не смогли создать поддеревья из-за ограничения листьев
        if left is None or right is None:
            self.leafs_cnt += 1
            return {'value': self._most_common_label(y)}

        return {
            'feature': col_name,
            'threshold': split_value,
            'left': left,
            'right': right
        }

    def get_best_split(self, X, y):
     best_ig = -1
     best_col = None
     best_threshold = None

     for col in X.columns:
        X_col = X[col].values
        unique_values = np.unique(X_col)

        # Проверяем все возможные пороги между уникальными значениями
        for i in range(len(unique_values) - 1):
            threshold = (unique_values[i] + unique_values[i+1]) / 2

            left_mask = X_col <= threshold
            right_mask = ~left_mask

            # Пропускаем если одна из сторон пустая
            if sum(left_mask) == 0 or sum(right_mask) == 0:
                continue

            # Вычисляем информационный прирост
            current_ig = self._information_gain(y, X_col, threshold)

            if current_ig > best_ig:
                best_ig = current_ig
                best_col = col
                best_threshold = threshold

     return best_col, best_threshold, best_ig if best_ig > 0 else 0

    def _information_gain(self, y, X_col, split_thresh):
        parent_entropy = self._entropy(y)
        left_mask = X_col <= split_thresh
        right_mask = ~left_mask
        if sum(left_mask) == 0 or sum(right_mask) == 0:
            return 0
        n = len(y)
        n_l, n_r = sum(left_mask), sum(right_mask)
        e_l, e_r = self._entropy(y[left_mask]), self._entropy(y[right_mask])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r
        ig = parent_entropy - child_entropy
        return ig

    def _entropy(self, y):
        counts = np.bincount(y)
        ps = counts / len(y)
        return -np.sum([p * np.log2(p) for p in ps if p > 0])

    def _most_common_label(self, y):
        return Counter(y).most_common(1)[0][0]

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for _, x in X.iterrows()])

    def _traverse_tree(self, x, node):
        if node is None:
            return 0
        if 'value' in node:
            return node['value']
        if x[node['feature']] <= node['threshold']:
            return self._traverse_tree(x, node['left'])
        else:
            return self._traverse_tree(x, node['right'])

    def print_tree(self, node=None, indent="", direction=None):
        if node is None:
            node = self.root

        if 'value' in node:
            if direction == 'left':
                print(f"{indent}leaf_left = {node['value']}")
            elif direction == 'right':
                print(f"{indent}leaf_right = {node['value']}")
            else:
                print(f"{indent}leaf = {node['value']}")
            return

        print(f"{indent}{node['feature']} > {node['threshold']}")

        if 'left' in node:
            self.print_tree(node['left'], indent + "  ", 'left')
        if 'right' in node:
            self.print_tree(node['right'], indent + "  ", 'right')

from sklearn.datasets import make_classification

X, y = make_classification(n_samples=150, n_features=5, n_informative=3, random_state=42)
X = pd.DataFrame(X).round(2)
y = pd.Series(y)
X.columns = [f'col_{col}' for col in X.columns]

In [50]:
X, y = make_classification(n_samples=150, n_features=5, n_informative=3, random_state=42)
X = pd.DataFrame(X).round(2)
y = pd.Series(y)
X.columns = [f'col_{col}' for col in X.columns]

In [51]:
cl = MyTreeClf(max_depth=7, min_samples_split=2)
cl.fit(X, y)
cl.print_tree()

col_0 > -0.1
  col_3 > 1.51
    leaf_left = 1
    col_1 > 1.445
      col_2 > -0.65
        col_3 > 1.935
          leaf_left = 1
          leaf_right = 0
        col_4 > -0.07
          col_0 > -1.27
            leaf_left = 1
            leaf_right = 0
          leaf_right = 1
      leaf_right = 1
  col_2 > -0.235
    leaf_left = 1
    leaf_right = 0


In [52]:
cl = MyTreeClf(max_depth=1, min_samples_split=1, max_leafs= 2)
cl.fit(X, y)
cl.print_tree()

col_0 > -0.1
  leaf_left = 1
  leaf_right = 0


In [53]:
cl = MyTreeClf(max_depth=3, min_samples_split=2, max_leafs= 5)
cl.fit(X, y)
cl.print_tree()

col_0 > -0.1
  col_3 > 1.51
    leaf_left = 1
    col_1 > 1.445
      leaf_left = 0
      leaf_right = 1
  col_2 > -0.235
    leaf_left = 1
    leaf_right = 0


In [ ]:
                       [1: col_0 > -0.1]
                      /                \
           [1.1: col_3 > 1.51]        [1.2: col_2 > -0.235]
          /              \              /              \
1.1.left (1.0)  [1.1.2: col_1 > 1.445]   1.2.left (1.0)   1.2.right (0.0)
                     /          \
       [1.1.2.1: col_2 > -0.65]   1.1.2.right (1.0)
           /               \
[1.1.2.1.1: col_3 > 1.935] [1.1.2.1.2: col_4 > -0.07]
    /            \               /            \
1.1.2.1.1.left 1.1.2.1.1.right 1.1.2.1.2.left 1.1.2.1.2.right
    (1.0)          (0.0)       (≈0.1429)         (1.0)

In [ ]:
                       [col_0 > -0.1]
                      /               \
          [col_3 > 1.51]              [col_2 > -0.235]
         /            \               /              \
  leaf_left=1    [col_1 > 1.445]   leaf_left=1    leaf_right=0
                   /          \
           [col_2 > -0.65]    leaf_right=1
            /            \
   [col_3 > 1.935]    [col_4 > -0.07]
    /          \        /          \
leaf_left=1 leaf_right=0 [col_0 > -1.27] leaf_right=1
                         /          \
                   leaf_left=1  leaf_right=0

In [ ]:
        [col_0 > -0.100]
        /            \
    [col_3 > 1.510]  [col_2 > -0.235]
    /          \        /        \
1.1.1.left  1.1.2  1.2.1.left 1.2.2.right
 (1)       /    \      (1)        (0)
      [col_1 > 1.445]
       /          \
1.1.2.1.left 1.1.2.2.right
     (0)          (1)